In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import (
    f1_score, 
    precision_score, 
    recall_score, 
    accuracy_score, 
    classification_report
)
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pickle

In [2]:
data = pd.read_pickle("/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/plbart_base_embeddings/processed_embeddings/plbart_embed_line_by_line_sum.pkl")
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,710.205383,-799.026489,321.919617,-1624.527222,2408.093262,-2533.795166,-817.205566,-702.485046,-464.826233,-554.832153,...,-90.683929,149.631775,-2021.500854,-3153.117676,-2010.092651,1909.142456,1098.878174,1,8077,critical
1,32.499386,-1.515829,-24.590275,-30.602459,47.787666,-27.378401,-21.693262,-6.852709,-29.279053,-34.747868,...,-26.822235,2.004940,-24.598465,-17.089935,-55.195568,30.086657,-24.063192,1,5553,critical
2,124.713303,-38.588451,-42.579781,-39.911690,85.068405,-40.210751,-44.931362,6.869379,-48.874641,-76.863480,...,45.405861,45.218468,-40.648548,-134.478836,-97.419083,32.619202,46.201210,1,9341,critical
3,130.854202,-74.683624,18.322701,-92.782379,122.927406,-85.169540,-60.637974,-51.506363,-53.193283,-14.298395,...,-42.749687,43.243038,-101.961884,-263.908386,-107.037132,140.791351,59.563663,1,10419,critical
4,710.205383,-799.026489,321.919617,-1624.527222,2408.093262,-2533.795166,-817.205566,-702.485046,-464.826233,-554.832153,...,-90.683929,149.631775,-2021.500854,-3153.117676,-2010.092651,1909.142456,1098.878174,1,12232,critical


In [3]:
# Separate features and labels
X = data[[col for col in data.columns if col.startswith("emb_")]].values  # Select embedding columns
y = data['label'].values 

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define classifiers
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
bagging = BaggingClassifier(estimator=RandomForestClassifier(), n_estimators=100, random_state=42)  
xgboost = xgb.XGBClassifier(n_estimators=100, random_state=42)

# Add the classifiers to a dictionary
classifiers = {
    "Random Forest": random_forest,
    "Bagging": bagging,
    "XGBoost": xgboost
}

In [4]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store cross-validation results
results = {}

# Perform cross-validation for each classifier
for clf_name, clf in classifiers.items():
    metrics_summary = {
        "f1_micro": [],
        "f1_macro": [],
        "accuracy": [],
        "precision_macro": [],
        "recall_macro": []
    }
    
    for train_idx, val_idx in skf.split(X_train, y_train):
        # Split the training data into training and validation folds
        X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        # Train the classifier
        clf.fit(X_train_fold, y_train_fold)
        
        # Make predictions on the validation set
        y_pred = clf.predict(X_val_fold)
        
        # Calculate metrics
        metrics_summary["f1_micro"].append(f1_score(y_val_fold, y_pred, average='micro'))
        metrics_summary["f1_macro"].append(f1_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["accuracy"].append(accuracy_score(y_val_fold, y_pred))
        metrics_summary["precision_macro"].append(precision_score(y_val_fold, y_pred, average='macro'))
        metrics_summary["recall_macro"].append(recall_score(y_val_fold, y_pred, average='macro'))
    
    # Store mean metrics for the classifier
    results[clf_name] = {metric: np.mean(scores) for metric, scores in metrics_summary.items()}

# Print cross-validation results
print("\nCross-Validation Results:")
for clf_name, metrics in results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")


Cross-Validation Results:

Random Forest Metrics:
F1_micro: 0.8309
F1_macro: 0.7748
Accuracy: 0.8309
Precision_macro: 0.7877
Recall_macro: 0.7653

Bagging Metrics:
F1_micro: 0.8371
F1_macro: 0.7754
Accuracy: 0.8371
Precision_macro: 0.8029
Recall_macro: 0.7588

XGBoost Metrics:
F1_micro: 0.8329
F1_macro: 0.7780
Accuracy: 0.8329
Precision_macro: 0.7896
Recall_macro: 0.7694


In [5]:
# Final evaluation on the test set
final_results = {}

for clf_name, clf in classifiers.items():
    # Train the classifier on the entire training set
    clf.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_test = clf.predict(X_test)
    
    # Calculate test metrics
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    accuracy_test = accuracy_score(y_test, y_pred_test)
    report_test = classification_report(y_test, y_pred_test)
    
    # Save final metrics for the classifier
    final_results[clf_name] = {
        "f1_micro": f1_micro_test,
        "f1_macro": f1_macro_test,
        "precision_macro": precision_test,
        "recall_macro": recall_test,
        "accuracy": accuracy_test,
        "classification_report": report_test
    }

# Print final test results
print("\nFinal Test Results:")
for clf_name, metrics in final_results.items():
    print(f"\n{clf_name} Metrics:")
    for metric, value in metrics.items():
        if metric != "classification_report":
            print(f"{metric.capitalize()}: {value:.4f}")
    print(f"Classification Report:\n{metrics['classification_report']}")


Final Test Results:

Random Forest Metrics:
F1_micro: 0.8215
F1_macro: 0.7652
Precision_macro: 0.7733
Recall_macro: 0.7584
Accuracy: 0.8215
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       814
           1       0.68      0.62      0.65       295

    accuracy                           0.82      1109
   macro avg       0.77      0.76      0.77      1109
weighted avg       0.82      0.82      0.82      1109


Bagging Metrics:
F1_micro: 0.8242
F1_macro: 0.7657
Precision_macro: 0.7784
Recall_macro: 0.7559
Accuracy: 0.8242
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       814
           1       0.69      0.61      0.65       295

    accuracy                           0.82      1109
   macro avg       0.78      0.76      0.77      1109
weighted avg       0.82      0.82      0.82      1109


XGBoost Metrics:
F1_micro: 0.8179
F1_macro: 

In [6]:
output_path = "/Users/mac/Desktop/Code_Smell_Detection/data_class/results/plbart/result_plbart_sum.pkl"
with open(output_path, "wb") as f:
    pickle.dump({"cross_val_results": results, "final_results": final_results}, f)
print(f"\nResults saved to {output_path}")


Results saved to /Users/mac/Desktop/Code_Smell_Detection/data_class/results/plbart/result_plbart_sum.pkl
